<a href="https://colab.research.google.com/github/lmunozad/QueryLab/blob/main/cudf_pandas_colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Acercamiento al modo acelerador de pandas de RAPIDS cuDF (cudf.pandas)

cuDF es una librería Python GPU DataFrame (construida sobre el formato de memoria columnar Apache Arrow) para cargar, unir, agregar, filtrar y manipular datos tabulares usando una API estilo DataFrame al estilo de pandas.

cuDF proporciona ahora un modo acelerador de pandas (`cudf.pandas`), permitiéndole llevar la computación acelerada a sus flujos de trabajo con pandas sin requerir ningún cambio en el código.

Este cuaderno es una breve introducción a `cudf.pandas`.


Adaptación y traducción desde (`rapidsai-community`,`https://alphasignal.ai/`)

# ⚠️ Verifica tu configuración

En primer lugar, comprobaremos que estás utilizando una GPU NVIDIA.

In [ ]:
!nvidia-smi  # muestra información sobre las GPUs disponibles

Sat May 25 14:27:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import cudf  # esto debería funcionar sin errores

# Descargar los datos

Los datos con los que trabajaremos son el conjunto de datos [Parking Violations Issued - Fiscal Year 2022](https://data.cityofnewyork.us/City-Government/Parking-Violations-Issued-Fiscal-Year-2022/7mxj-7a6y) de NYC Open Data.

Estamos descargando una copia de este conjunto de datos de un bucket s3 alojado por NVIDIA para proporcionar velocidades de descarga más rápidas. Comenzaremos descargando los datos. Tardaremos unos 30 segundos.

## Licencia y condiciones de los datos
Como este conjunto de datos procede del NYC Open Data Portal, se rige por su licencia y condiciones de uso.

### ¿Existen restricciones sobre cómo puedo utilizar los Datos Abiertos?

> Los Datos Abiertos pertenecen a todos los neoyorquinos. No hay restricciones en el uso de los Datos Abiertos. Consulte las Condiciones de uso para obtener más información.

### [Condiciones de uso](https://opendata.cityofnewyork.us/overview/#termsofuse)

> Al acceder a los conjuntos de datos y fuentes disponibles a través de NYC Open Data, el usuario acepta todas las Condiciones de uso de NYC.gov, así como la Política de privacidad de NYC.gov. El usuario también acepta las condiciones de uso adicionales definidas por los organismos, oficinas y oficinas que proporcionan los datos. Los conjuntos de datos públicos disponibles en NYC Open Data se proporcionan con fines informativos. El Ayuntamiento no garantiza la integridad, exactitud, contenido o idoneidad para cualquier propósito o uso particular de cualquier conjunto de datos públicos disponibles en NYC Open Data, ni se implican o infieren tales garantías con respecto a los conjuntos de datos públicos proporcionados.

> El Ayuntamiento no se hace responsable de ningún defecto de

In [ ]:
!wget https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet -O /tmp/nyc_parking_violations_2022.parquet

--2024-05-25 14:27:48--  https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet
Resolving data.rapids.ai (data.rapids.ai)... 99.86.38.104, 99.86.38.102, 99.86.38.71, ...
Connecting to data.rapids.ai (data.rapids.ai)|99.86.38.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 474211285 (452M) [binary/octet-stream]
Saving to: ‘/tmp/nyc_parking_violations_2022.parquet’

/tmp/nyc_parking_vi 100%[===================>] 452.24M  52.1MB/s    in 6.8s    

2024-05-25 14:27:55 (66.5 MB/s) - ‘/tmp/nyc_parking_violations_2022.parquet’ saved [474211285/474211285]



# Análisis usando Pandas estándar

En primer lugar, vamos a utilizar Pandas para leer algunas columnas del conjunto de datos:

In [ ]:
import pandas as pd

In [ ]:
# leer 5 columnas de los datos
df = pd.read_parquet(
    "/tmp/nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"]
)

# visualización de un ejemplo aleatorio de 10 filas
df.sample(10)

,Registration State,Violation Description,Vehicle Body Type,Issue Date,Summons Number
14771250,NY,PHTO SCHOOL ZN SPEED VIOLATION,SUBN,06/24/2022,4784376586
8108364,NY,71A-Insp Sticker Expired (NYS),2DSD,01/01/2022,8854995563
1962792,NY,PHTO SCHOOL ZN SPEED VIOLATION,4DSD,08/09/2021,4743732293
5629838,NY,21-No Parking (street clean),SUBN,11/06/2021,8900403448
3809193,NJ,38-Failure to Dsplay Meter Rec,SUBN,09/04/2021,8937366538
844779,NJ,38-Failure to Dsplay Meter Rec,4DSD,07/01/2021,8966470373
13466824,PA,21-No Parking (street clean),4DSD,04/28/2022,8582448946
14971798,NY,47-Double PKG-Midtown(Com Veh),VAN,06/22/2022,8824685043
9938315,WA,38-Failure to Dsplay Meter Rec,PICK,02/22/2022,8957615817
1305252,NY,71A-Insp Sticker Expired (NYS),SUBN,07/07/2021,8972973210


Ahora trataremos de responder algunas preguntas usando los datos

## ¿Qué infracción de aparcamiento cometen con más frecuencia los vehículos de varios estados de EE.UU.?

Cada registro de nuestro conjunto de datos contiene el estado de matriculación del vehículo infractor y el tipo de infracción de aparcamiento. Supongamos que queremos obtener el tipo de infracción más común para los vehículos registrados en diferentes estados. Podemos hacerlo en Pandas utilizando una combinación de [value_counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) y [GroupBy.head](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.head.html):

In [ ]:
(df[["Registration State", "Violation Description"]]  # obtener estas dos columnas
 .value_counts()  # obtener el recuento de infracciones por Estado y por tipo de infracción
 .groupby("Registration State")  # agrupar por estado
 .head(1)  # obtener la primera fila en cada grupo (el tipo de infracción con mayor recuento)
 .sort_index()  # ordenar por nombre del estado
 .reset_index() # iniciar nuevo indice
)

,Registration State,Violation Description,count
0,99,<NA>,17550
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
62,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
63,WA,21-No Parking (street clean),3732
64,WI,14-No Standing,1639
65,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


El código anterior utiliza [encadenamiento de métodos](https://tomaugspurger.net/posts/method-chaining/) para combinar una serie de operaciones en una única sentencia. Puede que te resulte útil dividir el código en varias sentencias e inspeccionar cada uno de los resultados intermedios.

## ¿Qué tipos de carrocería aparecen con más frecuencia en las infracciones de aparcamiento?

También podemos investigar qué tipos de carrocería de vehículos aparecen con más frecuencia en las infracciones de aparcamiento

In [ ]:
(df
 .groupby(["Vehicle Body Type"])
 .agg({"Summons Number": "count"})
 .rename(columns={"Summons Number": "Count"})
 .sort_values(["Count"], ascending=False)
)

,Count
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
XRL,1
XT,1
YANT,1


## ¿Cómo varían las infracciones de aparcamiento según los días de la semana?

In [ ]:
weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

issue_weekday
Sunday        462992
Saturday     1108385
Monday       2488563
Wednesday    2760088
Tuesday      2809949
Friday       2891679
Thursday     2913951
Name: Summons Number, dtype: int64

Parece que hay menos infracciones los fines de semana, ¡lo cual tiene sentido! Durante la semana, hay más gente conduciendo en Nueva York.

## ¡Vamos a cronometrarlo!

Cargar y procesar estos datos nos ha llevado un poco de tiempo. Midamos cuánto tardan estos pipelines en Pandas:

In [ ]:
%%time

# Lectura de los datos tipo parquet
df = pd.read_parquet(
    "/tmp/nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"]
)

# Consulta con encadenamiento de métodos - infracción con más frecuencia por estado


(df[["Registration State", "Violation Description"]]
 .value_counts()
 .groupby("Registration State")
 .head(1)
 .sort_index()
 .reset_index()
)

CPU times: user 5.67 s, sys: 2.11 s, total: 7.78 s
Wall time: 6.59 s


,Registration State,Violation Description,count
0,99,<NA>,17550
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
62,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
63,WA,21-No Parking (street clean),3732
64,WI,14-No Standing,1639
65,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [ ]:
%%time

# Tipos de carroceria más frecuentes en infracciones
(df
 .groupby(["Vehicle Body Type"])
 .agg({"Summons Number": "count"})
 .rename(columns={"Summons Number": "Count"})
 .sort_values(["Count"], ascending=False)
)

CPU times: user 746 ms, sys: 221 ms, total: 966 ms
Wall time: 963 ms


,Count
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
XRL,1
XT,1
YANT,1


In [ ]:
%%time

# Variación de las infracciones por dias de la semana

weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

CPU times: user 3.85 s, sys: 554 ms, total: 4.41 s
Wall time: 4.81 s


issue_weekday
Sunday        462992
Saturday     1108385
Monday       2488563
Wednesday    2760088
Tuesday      2809949
Friday       2891679
Thursday     2913951
Name: Summons Number, dtype: int64

# Usando cudf.pandas

Ahora, vamos a volver a ejecutar el código Pandas anterior con la extensión `cudf.pandas` cargada.

Típicamente, deberías cargar la extensión `cudf.pandas` como primer paso en tu notebook, antes de importar cualquier módulo. Aquí, reiniciamos explícitamente el kernel para simular ese comportamiento.

In [ ]:
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [ ]:
%load_ext cudf.pandas

In [ ]:
%%time

import pandas as pd

df = pd.read_parquet(
    "/tmp/nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"]
)

(df[["Registration State", "Violation Description"]]
 .value_counts()
 .groupby("Registration State")
 .head(1)
 .sort_index()
 .reset_index()
)

CPU times: user 647 ms, sys: 319 ms, total: 966 ms
Wall time: 1.31 s


,Registration State,Violation Description,count
0,99,<NA>,17550
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
62,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
63,WA,21-No Parking (street clean),3732
64,WI,14-No Standing,1639
65,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [ ]:
%%time

(df
 .groupby(["Vehicle Body Type"])
 .agg({"Summons Number": "count"})
 .rename(columns={"Summons Number": "Count"})
 .sort_values(["Count"], ascending=False)
)

CPU times: user 20.5 ms, sys: 6.88 ms, total: 27.3 ms
Wall time: 44.3 ms


,Count
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
YANT,1
YBSD,1
YEL,1


In [ ]:
%%time

weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

CPU times: user 242 ms, sys: 74.2 ms, total: 317 ms
Wall time: 395 ms


issue_weekday
Sunday        462992
Saturday     1108385
Monday       2488563
Wednesday    2760088
Tuesday      2809949
Friday       2891679
Thursday     2913951
Name: Summons Number, dtype: int64

¡Mucho más rápido! Las operaciones que tardaban entre 5 y 20 segundos ahora pueden terminar en milisegundos sin cambiar el código.

# Comprendiendo el Rendimiento

`cudf.pandas` proporciona utilidades de perfilado para ayudarte a entender mejor el rendimiento. Con estas herramientas, puedes identificar qué partes de tu código se ejecutan en la GPU y qué partes se ejecutan en la CPU.

Son accesibles en el espacio de nombres `cudf.pandas` desde que la extensión `cudf.pandas` fue cargada arriba con `load_ext cudf.pandas`.

#### Nota de Colab
Si está ejecutando en Colab, la primera vez que utilice el perfilador puede tardar más de 10 segundos debido a que el depurador de Colab interactúa con la función incorporada de Python [sys.settrace](https://docs.python.org/3/library/sys.html#sys.settrace) que utilizamos para el perfilado. Para fines de demostración, esto no es un problema. Simplemente ejecute la celda de nuevo.

## Perfilando la Funcionalidad

Podemos generar un perfil por función:

In [ ]:
len(df)

15435607

In [ ]:
%%cudf.pandas.profile

small_df = pd.DataFrame({'a': ["0", "1", "2"], 'b': ["x", "y", "z"]})
small_df = pd.concat([small_df, small_df])

axis = 0
for i in range(0, 2):
    small_df.min(axis=axis)
    axis = i

counts = small_df.groupby("a").b.count()

In [ ]:
%%cudf.pandas.line_profile

small_df = pd.DataFrame({'a': ["0", "1", "2"], 'b': ["x", "y", "z"]})
small_df = pd.concat([small_df, small_df])

axis = 0
for i in range(0, 2):
    small_df.min(axis=axis)
    axis = 1

counts = small_df.groupby("a").b.count()


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/cudf/pandas/profiler.py", line 97, in __enter__
    sys.settrace(self._tracefunc)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/cudf/pandas/profiler.py", line 116, in __exit__
    sys.settrace(self._oldtrace)



                                                                                                                   
                                             Total time elapsed: 2.646 seconds                                     
                                                                                                                   
                                                           Stats                                                   
                                                                                                                   
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Line no. ┃ Line                                                                     ┃ GPU TIME(s) ┃ CPU TIME(s) ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 2        │     small_df = pd.DataFrame({'a': ["0", "1", "2"], 'b': ["x", "y", "z"]… │ 0.004195585 │             │
│          │                                                                          │             │             │
│ 3        │     small_df = pd.concat([small_df, small_df])                           │ 0.008323235 │             │
│          │                                                                          │             │             │
│ 5        │     axis = 0                                                             │             │             │
│          │                                                                          │             │             │
│ 6        │     for i in range(0, 2):                                                │             │             │
│          │                                                                          │             │             │
│ 7        │         small_df.min(axis=axis)                                          │ 0.007513770 │ 0.033017991 │
│          │                                                                          │             │             │
│ 8        │         axis = 1                                                         │             │             │
│          │                                                                          │             │             │
│ 10       │     counts = small_df.groupby("a").b.count()                             │ 0.019637824 │             │
│          │                                                                          │             │             │
└──────────┴──────────────────────────────────────────────────────────────────────────┴─────────────┴─────────────┘

## Entre bastidores: ¿Qué está pasando aquí?

Cuando cargas `cudf.pandas`, los tipos de Pandas como `Series` y `DataFrame` son reemplazados por objetos proxy que despachan operaciones a cuDF cuando es posible. Podemos verificar que `cudf.pandas` está activo mirando nuestra variable `pd`:

In [ ]:
pd

<module 'pandas' (ModuleAccelerator(fast=cudf, slow=pandas))>

Como resultado, todas las funciones, métodos y objetos creados por pandas son proxies:

In [ ]:
type(pd.read_csv)

cudf.pandas.fast_slow_proxy._FunctionProxy

Las operaciones soportadas por cuDF serán **muy** rápidas:

In [ ]:
%%time
df.count(axis=0)

CPU times: user 1.93 ms, sys: 0 ns, total: 1.93 ms
Wall time: 1.94 ms


Registration State       15435607
Violation Description    15117819
Vehicle Body Type        15402365
Issue Date               15435607
Summons Number           15435607
issue_weekday            15435607
dtype: int64

Las operaciones no soportadas por cuDF serán más lentas, ya que vuelven a utilizar Pandas (copiando datos entre la CPU y la GPU según sea necesario). Por ejemplo, cuDF no soporta actualmente el parámetro `axis=` del método `count`. Así que esta operación se ejecutará en la CPU y será notablemente más lenta que la anterior.

In [ ]:
%%time
df.count(axis=1) # Esto usará pandas, porque cuDF no soporta axis=1 para el método .count()

CPU times: user 8.36 s, sys: 2.25 s, total: 10.6 s
Wall time: 13.6 s


0           5
1           5
2           5
3           5
4           5
           ..
15435602    6
15435603    6
15435604    6
15435605    6
15435606    6
Length: 15435607, dtype: int64

Pero la historia no acaba aquí. A menudo necesitamos mezclar nuestro propio código con librerías de terceros que otras personas han escrito. Muchas de estas bibliotecas aceptan objetos pandas como entradas.

# Usando librerias de terceros con cudf.pandas

Puedes pasar objetos Pandas a librerías de terceros cuando uses `cudf.pandas`, igual que harías cuando usas Pandas normal.

A continuación, mostramos un ejemplo de uso de [plotly-express](https://plotly.com/python/plotly-express/) para visualizar los datos que hemos estado procesando:

## ¿Visualizar qué estados tienen más camionetas en relación con otros vehículos?

In [ ]:
import plotly.express as px

df = df.rename(columns={
    "Registration State": "reg_state",
    "Vehicle Body Type": "vehicle_type",
})

# recuentos de vehículos por estado:
counts = df.groupby("reg_state").size().sort_index()
# vehículos con tipo "PICK" (Pickup Truck)
pickup_counts = df.where(df["vehicle_type"] == "PICK").groupby("reg_state").size()
# porcentaje de camionetas por estado:
pickup_frac = ((pickup_counts / counts) * 100).rename("% Pickup Trucks")
del pickup_frac["MB"]  # (¡Manitoba es un caso atípico!)
# trazar los resultados:
pickup_frac = pickup_frac.reset_index()
px.choropleth(pickup_frac, locations="reg_state", color="% Pickup Trucks", locationmode="USA-states", scope="usa")

## Beyond just passing data: **Acelerando** el código de terceros

Poder pasar estos objetos proxy a librerías como Plotly es genial, pero los beneficios no acaban ahí.

Cuando se activa `cudf.pandas`, las operaciones de pandas que se ejecutan **dentro de las funciones de la librería de terceros** también se beneficiarán de la aceleración de la GPU cuando sea posible.

Abajo, puedes ver una imagen que ilustra cómo `cudf.pandas` puede acelerar el backend de pandas en Ibis, una librería que proporciona una API DataFrame unificada a varios backends. Hemos ejecutado este ejemplo en un sistema con una GPU NVIDIA H100 y una CPU Intel Xeon Platinum 8480CL.


Al cargar la extensión `cudf.pandas`, las operaciones de pandas dentro de Ibis pueden utilizar la GPU sin necesidad de cambiar el código. Simplemente funciona.

![ibis](https://drive.google.com/uc?id=1uOJq2JtbgVb7tb8qw8a2gG3JRBo72t_H)

# Conclusión

Con `cudf.pandas`, puedes seguir usando pandas como tu librería principal de dataframes. Cuando las cosas empiecen a ir un poco lentas, ¡simplemente carga `cudf.pandas` y ejecuta tu código existente en una GPU!

Para obtener más información, te animamos a visitar [rapids.ai/cudf-pandas](https://rapids.ai/cudf-pandas).